<a href="https://colab.research.google.com/github/emiliomercuri/MixturesMonteCarlo/blob/main/notebooks/Exemplo_MC%2BCurvas_granulome%CC%81tricas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulação Monte Carlo para misturas de 4 materiais

Código Python para calcular o menor somatório de RSS de misturas ternárias em relação à relação de Alfred (Funk and Dinger, 1980).

## Modelo para predição do empacotamento de partículas

Funk and Dinger (1980): q = 0.37 (maximum packing density: spheres)

Cumulative Percentage Finer Then (CPFT) $d$

$$
\text{CPFT}= \frac{d^q-D_{\rm Min}^q}{D_{\rm Max}^q-D_{\rm Min}^q}
$$

Where:

* $d$: particle diameter (sieve openings) (mm);
* $D_{\rm Min}$: minimum particle diameter (mm);
* $D_{\rm Max}$: maximum particle diameter (mm);
* $q$: distribution modulus.


In [5]:
# Importando bibliotecas
import pandas as pd
import numpy
import random

### Importanto os CSVs

In [6]:
dt = pd.read_csv('https://raw.githubusercontent.com/emiliomercuri/MixturesMonteCarlo/main/data/dados_dosagem_mistura.csv',sep=',')

In [9]:
dt[0:3]

,diametro(mm),Cimento,Areia,Pedrisco,Brita
0,25.0,100.0,100.0,100.00,98.82
1,19.0,100.0,100.0,100.00,80.69
2,12.5,100.0,100.0,99.89,27.38


In [11]:
# Pode mudar o expoente aqui:
q = 0.25

diametro   = dt['diametro(mm)'].values
cimento    = dt['Cimento'].values
areia      = dt['Areia'].values
pedrisco   = dt['Pedrisco'].values
brita      = dt['Brita'].values

In [14]:
# 0 = cimento
# 1 = areia
# 2 = pedrisco
# 3 = brita

In [20]:
proporcoes = numpy.random.uniform(0,1.0,size=3)
proporcoes

array([0.03341356, 0.23983805, 0.44677944])

In [22]:
1-proporcoes.sum()

0.27996894457262456

In [28]:
l_p_ci = []
l_p_ar = []
l_p_pe = []
l_p_br = []

n = 0 
while n < 100000:
  proporcoes = numpy.random.uniform(0,1.0,size=3)
  if proporcoes.sum()<1.0:
    #if proporcoes[1] >= 0.3*proporcoes[0]:
      n += 1
      proporcoes_lista = list(proporcoes)
      proporcoes_lista.append(1-proporcoes.sum())
      prop = numpy.array(proporcoes_lista)
      #print(proporcoes_lista, prop.sum())
      
      # cimento
      l_p_ci.append(proporcoes_lista[0])

      # areia
      l_p_ar.append(proporcoes_lista[1])

      # pedrisco
      l_p_pe.append(proporcoes_lista[2])

      # brita
      l_p_br.append(proporcoes_lista[3])         

In [25]:
#l_p_ci

### Simulação Monte Carlo - Mistura de 4 materiais

In [29]:
l_soma_rss = []
    
for k in range(100000):

    conjunto = cimento*l_p_ci[k] + areia*l_p_ar[k] + pedrisco*l_p_pe[k] + brita*l_p_br[k]
    alfred = 100*(((diametro**q)-(numpy.min(diametro))**q)/((numpy.max(diametro)**q)-(numpy.min(diametro)**q)))
    rss = ((conjunto/100)-(alfred/100))**2 # residual squares
    soma_rss = numpy.sum(rss) # residual sum of squares
      
    l_soma_rss.append(soma_rss)

posicao = numpy.where(numpy.array(l_soma_rss) == numpy.min(l_soma_rss))
pos = posicao[0][0]    

In [30]:
print("O menor somatório RSS é: ", l_soma_rss[pos])
print(" ")
print("Proporção cimento: ", l_p_ci[pos])
print("Proporção areia: ",   l_p_ar[pos])
print("Proporção pedrisco: ",l_p_pe[pos])
print("Proporção brita: ",   l_p_br[pos])

O menor somatório RSS é:  0.018445101670306514
 
Proporção cimento:  0.10478717325704223
Proporção areia:  0.41038472560391004
Proporção pedrisco:  0.253765640827722
Proporção brita:  0.2310624603113257


O menor somatório RSS é:  0.0186386110013631
 
Proporção cimento:  0.10224702562608012
Proporção areia:  0.42732158036126056
Proporção pedrisco:  0.22822015931563655
Proporção brita:  0.24221123469702277